In [115]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical

from keras import backend as K 


In [116]:
from PIL import Image # used for loading images
import numpy as np
import os # used for navigating to image path
import imageio # used for writing images
import cv2


In [117]:
def size_stat(DIR):
  heights = []
  widths = []
  ct = 0
  for catagory in os.listdir(DIR):
    folder = os.path.join(DIR, catagory)
    for img in os.listdir(folder):
        #print(img)
        path = os.path.join(folder, img)
        data = np.array(Image.open(path)) #PIL Image library
        heights.append(data.shape[0])
        widths.append(data.shape[1])
        ct += 1
  avg_height = sum(heights) / len(heights)
  avg_width = sum(widths) / len(widths)
  print("Totoal images: " + str(ct) + '\n')
  print("Average Height: " + str(avg_height))
  print("Max Height: " + str(max(heights)))
  print("Min Height: " + str(min(heights)))
  print('\n')
  print("Average Width: " + str(avg_width))
  print("Max Width: " + str(max(widths)))
  print("Min Width: " + str(min(widths)))

In [118]:
size_stat('trainingData')

Totoal images: 450

Average Height: 27.202222222222222
Max Height: 206
Min Height: 7


Average Width: 27.586666666666666
Max Width: 181
Min Width: 10


In [119]:
img_width, img_height = 27, 27
  
train_data_dir = 'trainingData'
validation_data_dir = 'validationData'
nb_train_samples = 450 
nb_validation_samples = 50
epochs = 30
batch_size = 10

In [120]:
if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3)

In [121]:
train_datagen = ImageDataGenerator( rescale = 1. / 255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True) 

test_datagen = ImageDataGenerator(rescale = 1. / 255) 

In [122]:
train_generator = train_datagen.flow_from_directory(train_data_dir, 
                                                    target_size =(img_width, img_height), 
                                                    batch_size = batch_size, 
                                                    class_mode ='categorical') 
print(len(train_generator.filenames))
print(train_generator.class_indices)
print(len(train_generator.class_indices))
class_dictionary = train_generator.class_indices
num_classes = len(class_dictionary)

Found 450 images belonging to 5 classes.
450
{'ஃ': 0, 'இ': 1, 'க': 2, 'த': 3, 'ப': 4}
5


In [123]:
num_classes = len(train_generator.class_indices)

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

print(train_labels)

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [124]:
validation_generator = test_datagen.flow_from_directory(validation_data_dir, 
                                                        target_size =(img_width, img_height), 
                                                        batch_size = batch_size, class_mode ='categorical') 

Found 50 images belonging to 5 classes.


In [125]:
validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

In [126]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape = input_shape)) 
model.add(Activation('relu')) 
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [127]:
model.compile(loss ='categorical_crossentropy', optimizer ='rmsprop', metrics =['accuracy']) 

In [128]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        416       
_________________________________________________________________
activation_5 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 21632)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 256)               5538048   
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 5)                 1285      
Total params: 5,539,749
Trainable params: 5,539,749
Non-trainable params: 0
_________________________________________________________________


In [129]:
model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size, 
                    epochs = epochs, 
                    validation_data = validation_generator, 
                    validation_steps = nb_validation_samples // batch_size) 

Epoch 1/30
45/45 [==============================] - 9s 197ms/step - loss: 2.9604 - acc: 0.4867 - val_loss: 0.7637 - val_acc: 0.6800
Epoch 2/30
45/45 [==============================] - 7s 154ms/step - loss: 0.4733 - acc: 0.8378 - val_loss: 0.4524 - val_acc: 0.8000
Epoch 3/30
45/45 [==============================] - 7s 161ms/step - loss: 0.3136 - acc: 0.8956 - val_loss: 0.2591 - val_acc: 0.9000
Epoch 4/30
45/45 [==============================] - 7s 157ms/step - loss: 0.1938 - acc: 0.9378 - val_loss: 0.0876 - val_acc: 0.9800
Epoch 5/30
45/45 [==============================] - 7s 160ms/step - loss: 0.1276 - acc: 0.9578 - val_loss: 0.0776 - val_acc: 0.9800
Epoch 6/30
45/45 [==============================] - 7s 152ms/step - loss: 0.1144 - acc: 0.9578 - val_loss: 0.2114 - val_acc: 0.9200
Epoch 7/30
45/45 [==============================] - 7s 157ms/step - loss: 0.0792 - acc: 0.9756 - val_loss: 0.0912 - val_acc: 0.9400
Epoch 8/30
45/45 [==============================] - 7s 156ms/step - loss: 0.

In [130]:
model1 = Sequential()
model1.add(Conv2D(32, (2, 2), input_shape = input_shape)) 
model1.add(Activation('relu')) 
model1.add(Dense(50, activation='relu'))
model1.add(Flatten())
model1.add(Dense(50, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes, activation='softmax'))
model1.compile(loss ='categorical_crossentropy', optimizer ='rmsprop', metrics =['accuracy']) 
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        416       
_________________________________________________________________
activation_6 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
dense_22 (Dense)             (None, 26, 26, 50)        1650      
_________________________________________________________________
flatten_11 (Flatten)         (None, 33800)             0         
_________________________________________________________________
dense_23 (Dense)             (None, 50)                1690050   
_________________________________________________________________
dropout_11 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 5)                 255       
Total para

In [131]:
model1.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size, 
                    epochs = 20, 
                    validation_data = validation_generator, 
                    validation_steps = nb_validation_samples // batch_size) 

Epoch 1/20
45/45 [==============================] - 4s 78ms/step - loss: 1.6264 - acc: 0.4178 - val_loss: 1.1358 - val_acc: 0.5800
Epoch 2/20
45/45 [==============================] - 3s 60ms/step - loss: 0.9448 - acc: 0.6222 - val_loss: 0.6914 - val_acc: 0.7200
Epoch 3/20
45/45 [==============================] - 3s 57ms/step - loss: 0.7744 - acc: 0.7022 - val_loss: 0.4348 - val_acc: 0.8600
Epoch 4/20
45/45 [==============================] - 3s 57ms/step - loss: 0.5734 - acc: 0.7756 - val_loss: 0.2495 - val_acc: 0.9000
Epoch 5/20
45/45 [==============================] - 3s 57ms/step - loss: 0.5220 - acc: 0.7889 - val_loss: 0.1899 - val_acc: 0.9800
Epoch 6/20
45/45 [==============================] - 3s 56ms/step - loss: 0.4412 - acc: 0.8044 - val_loss: 0.0771 - val_acc: 0.9800
Epoch 7/20
45/45 [==============================] - 2s 55ms/step - loss: 0.3605 - acc: 0.8356 - val_loss: 0.2200 - val_acc: 0.9200
Epoch 8/20
45/45 [==============================] - 3s 56ms/step - loss: 0.3703 - a

In [145]:
    image_path = 'pa.jpeg'

    orig = cv2.imread(image_path)

    print("[INFO] loading and preprocessing image...")
    image = load_img(image_path, target_size=(27, 27))
    image = img_to_array(image)

    # important! otherwise the predictions will be '0'
    image = image / 255

    image = np.expand_dims(image, axis=0)

[INFO] loading and preprocessing image...


In [146]:
#prediction = model.predict_classes(image)
#print(prediction)
class_predicted = model.predict_classes(image)

probabilities = model.predict_proba(image)

print(probabilities)

inID = class_predicted[0]

inv_map = {v: k for k, v in class_dictionary.items()}

label = inv_map[inID]

# get the prediction label
print("Image ID: {}, Label: {}".format(inID, label))

[[2.1605244e-10 1.6281776e-05 1.4319778e-09 5.8122951e-09 9.9998367e-01]]
Image ID: 4, Label: ப


In [147]:
#prediction = model.predict_classes(image)
#print(prediction)
class_predicted = model1.predict_classes(image)

probabilities = model1.predict_proba(image)

print(probabilities)

inID = class_predicted[0]

inv_map = {v: k for k, v in class_dictionary.items()}

label = inv_map[inID]

# get the prediction label
print("Image ID: {}, Label: {}".format(inID, label))

[[3.0039211e-05 1.1486738e-07 2.4560780e-08 1.0766754e-03 9.9889320e-01]]
Image ID: 4, Label: ப
